In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

In [ ]:
dataset_path = '/content/drive/MyDrive/DIP/Shells_or_Pebbles'

In [ ]:
# Load images and labels from the dataset
def load_dataset():
    images = []
    labels = []
    for label in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label)
        for image_file in os.listdir(label_path):
            image_path = os.path.join(label_path, image_file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (64, 64))  # Resize the image to a desired size
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
# Extract color histogram features from images
def extract_color_histogram(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_image], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

In [ ]:
# Load the dataset
images, labels = load_dataset()

In [ ]:
labels

array(['Shells', 'Shells', 'Shells', ..., 'Pebbles', 'Pebbles', 'Pebbles'],
      dtype='<U7')

In [ ]:
# Extract color histogram features for each image
features = []
for image in images:
    hist = extract_color_histogram(image)
    features.append(hist)
features = np.array(features)

In [ ]:
# Reshape the features array to 2D
n_samples = features.shape[0]
features = features.reshape(n_samples, -1)

In [ ]:
#prev
print("Features shape:", features.shape)
print("Labels shape:", labels.shape)

Features shape: (4284, 512)
Labels shape: (4284,)


In [ ]:
#prev
y_train = (labels == 'Shells').astype(int)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
# Convert labels to binary format (0 or 1)
y_train = (y_train == 'Shells').astype(int)
y_test = (y_test == 'Pebbles').astype(int)

In [ ]:
# Reshape features for CNN input (convert 2D to 3D)
X_train = X_train.reshape(X_train.shape[0], 8, 8, 8, 1)
X_test = X_test.reshape(X_test.shape[0], 8, 8, 8, 1)

In [ ]:
#chatgpt
# Build the CNN model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(512,)))  # Assuming 512 histogram bins
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print("Accuracy:", accuracy)

In [ ]:
# Predict a single image
def predict_single_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (64, 64))  # Resize the image to a desired size
    hist = extract_color_histogram(image)
    hist = hist.reshape(1, -1)
    hist = hist.reshape(1, 8, 8, 8, 1)  # Reshape for CNN input
    prediction = model.predict(hist)
    if prediction[0][0] >= 0.5:
        return 'Shells'
    else:
        return 'Pebbles'

In [ ]:
# Path to the image you want to predict
image_to_predict = '/content/drive/MyDrive/DIP_Lab/Lab3/Shells_or_Pebbles/Pebbles/Pebbles (1).jpg'

# Predict the class of the single image
prediction = predict_single_image(image_to_predict)
print("Prediction:", prediction)